# Pop2Piano - Colab Workflow (GitHub Edition)

This notebook runs Pop2Piano using the code from your GitHub repository.

## Key Steps:
1. **Install Dependencies:** With fixes for environment issues.
2. **Mock Essentia:** Trick to make transformers work.
3. **Clone Repository:** Fetch your fixed code from GitHub.
4. **Download Data:** Run the download script.
5. **Inference:** Generate Piano Covers.

In [ ]:
# @title 1. Install Dependencies 📦
import os
import sys
import subprocess

print("Installing dependencies... ⏳")

# Install Python packages
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "git+https://github.com/huggingface/transformers.git"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "librosa", "pretty_midi", "midi2audio", "scipy", "yt-dlp", "imageio-ffmpeg", "omegaconf", "joblib", "tqdm"])

# Install System packages
os.system("sudo apt-get update -q")
os.system("sudo apt-get install -y -q fluidsynth fluid-soundfont-gm ffmpeg")

print("Done! ✅")

In [ ]:
# @title 2. Apply Magic Fixes (Essentia Mocking) 🪄
import sys
from unittest.mock import MagicMock
from importlib.machinery import ModuleSpec

# Fix for missing essentia
mock_essentia = MagicMock()
# Important fix for Python 3.12+ (spec must be set)
mock_essentia.__spec__ = ModuleSpec(name='essentia', loader=None)

sys.modules["essentia"] = mock_essentia
sys.modules["essentia.standard"] = MagicMock()

print("Essentia mocked successfully! 🧙‍♂️")

In [ ]:
# @title 3. Clone Repository 🐙
import os

repo_url = "https://github.com/kareemkamal10/pop2piano.git"
repo_name = "pop2piano"

if os.path.exists(repo_name):
    print("Repository already cloned. Pulling latest changes...")
    %cd {repo_name}
    !git pull
else:
    print(f"Cloning {repo_url}...")
    !git clone {repo_url}
    %cd {repo_name}

print(f"Current working directory: {os.getcwd()}")

In [ ]:
# @title 4. Run Download Script 📥
# Ensure you are in the 'pop2piano' directory (checked in previous step)

# Run download with 15GB size limit as requested
!python download/download.py train_dataset.csv output_dir/ --max_size_gb 15.0

# If you want unlimited, remove the --max_size_gb argument

In [ ]:
# @title 5. Run Inference 🎹
import torch
import librosa
from transformers import Pop2PianoForConditionalGeneration, AutoFeatureExtractor, AutoTokenizer
from google.colab import files
from midi2audio import FluidSynth
from IPython.display import Audio, display

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Working on: {device} 🚀")

model_id = "sweetcocoa/pop2piano"
model = Pop2PianoForConditionalGeneration.from_pretrained(model_id).to(device)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

print("Model loaded! ✅")

def process_audio(audio_path):
    audio, sr = librosa.load(audio_path, sr=44100)
    tempo, beat_frames = librosa.beat.beat_track(y=audio, sr=sr)
    inputs = feature_extractor(audio, sampling_rate=sr, return_tensors="pt").to(device)
    
    target_composer = "pop2piano_kpop_v1"
    with torch.no_grad():
        model_output = model.generate(input_features=inputs["input_features"], composer=target_composer)
    return model_output, inputs

print("\n>>> Upload your Song (MP3/WAV):")
uploaded = files.upload()

if uploaded:
    audio_path = next(iter(uploaded))
    try:
        model_output, inputs = process_audio(audio_path)
        
        # Use the cheated processor to save MIDI
        from transformers import Pop2PianoProcessor
        processor = Pop2PianoProcessor.from_pretrained(model_id)
        midi = processor.batch_decode(model_output, feature_extractor_output=inputs)
        
        midi_filename = "piano_output.mid"
        wav_filename = "piano_output.wav"
        processor.save_as_midi(midi, midi_filename)
        
        FluidSynth(sound_font="/usr/share/sounds/sf2/FluidR3_GM.sf2").midi_to_audio(midi_filename, wav_filename)
        
        print("\n--- 🎶 Result ---")
        display(Audio(wav_filename))
        files.download(midi_filename)
        files.download(wav_filename)
    except Exception as e:
        print(f"Error: {e}")